In [1]:
!pip install "ru_patents_ner @ https://huggingface.co/Delicalib/ru_patents_ner/resolve/main/ru_patents_ner-any-py3-none-any.whl"
!pip install "ru_patents_rel @ https://huggingface.co/Delicalib/ru_patents_rel/resolve/main/ru_patents_rel-any-py3-none-any.whl"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 619.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import random
import typer
import spacy

from itertools import permutations

from pathlib import Path
from spacy.tokens import DocBin, Doc

from spacy.training.example import Example
from spacy import displacy

loaded_ner_model = spacy.load("ru_patents_ner")
loaded_rel_model = spacy.load("ru_patents_rel")
loaded_rel_model.add_pipe('sentencizer')

In [5]:
# @title Текст заголовка по умолчанию
text = [
    "Вставка декоративная обивки двери салона автомобиля удлиненного прямолинейного контура коробообразной формы поперечного сечения, выполненная с возможностью фиксирования на обивке двери и креплением на ней устройства подсветки в сборе посредством крепежно-фиксирующих элементов, конструктивно выполненных в виде зацепов, приварных элементов, позиционирующих пинов, ответных отверстий и клипс, причем приварные элементы, равномерно сформированные по контуру тыльной стороны вставки декоративной, входят в зацепление с корпусом устройства подсветки через ответные отверстия, жестко соединяя их методом сварки, а зацепы, равноудаленно сформированные на наружной поверхности корпуса устройства подсветки и выполненные с усилительными ребрами по боковым кромкам на переднем участке, разнонаправленными под углом друг к другу и основной поверхности зацепов, входят в зацепление с ответными клипсами, предварительно установленными в поверхности соответствующей зоны обивки двери, эквидистантно расположению и количеству зацепов сопрягаемой поверхности, позиционируя и ограничивая перемещение декоративной вставки относительно обивки двери, позиционирующие пины, выполненные четырехгранными и расположенными на концевых участках наружной поверхности корпуса устройства подсветки, позиционируют декоративную вставку на обивке двери при монтаже, позиционирующий элемент типа «гарпун», расположенный по центру наружной поверхности корпуса устройства подсветки, фиксирует декоративную вставку на обивке двери, ограничивая перемещение декоративной вставки в направлении «на вылет» из посадочных мест.",
    "Изобретение относится к области беспроводной связи. Технический результат изобретения заключается в ускорении создания группы для связи устройств. Устройство связи содержит получающий модуль для получения первой информации состояния, представляющей состояние первого устройства беспроводной связи относительно прямого соединения между устройствами с использованием беспроводной связи, и второй информации состояния, представляющей состояние второго устройства беспроводной связи относительно такого прямого соединения, и модуль управления для установки соединения между первым устройством беспроводной связи и вторым устройством беспроводной связи с использованием беспроводной связи на основе первой информации состояния и второй информации состояния. По меньшей мере одну из первой информации состояния и второй информации состояния получают с использованием связи в ближней зоне. 3 н. и 19 з.п. ф-лы, 4 табл., 50 ил.",
    "Описана полоуборочная машина, в частности полоуборочная машина с ручным управлением и/или переносная полоуборочная машина, содержащая несущее устройство, по меньшей мере один приводимый во вращение и снабженный мягкой обшивкой чистящий валик, расположенный на несущем устройстве, вентиляторное устройство для генерирования потока всасываемой среды и потоконаправляющее устройство, предназначенное для направления потока всасываемой среды и образующее по меньшей мере один всасывающий канал, сообщающий вентиляторное устройство с по меньшей мере одним чистящим валиком, причем по меньшей мере один всасывающий канал имеет по меньшей мере один вход, обращенный к по меньшей мере одному чистящему валику и имеющий первую входную стенку и расположенную на расстоянии от нее вторую входную стенку, между которыми образовано входное отверстие, причем когда по меньшей мере один чистящий валик приложен к очищаемому полу, первая входная стенка расположена по направлению действия силы тяжести выше второй входной стенки, причем первая входная стенка и/или вторая входная стенка прилегает к мягкой обшивке по меньшей мере одного чистящего валика или вдается в нее. 43 з.п. ф-лы, 16 ил.",
    "Раскрывается стиральная машина, содержащая основной барабан, свободно вращающийся в баке, вспомогательный барабан, смонтированный внутри основного барабана, чтобы относительно вращаться по отношению к основному барабану, внешний вал для вращения основного барабана, внутренний вал для вращения вспомогательного барабана, расположенный внутри внешнего вала, и приводной двигатель. Приводное средство конфигурируется для вращения основного барабана и вспомогательного барабана в разных направлениях и/или с разными скоростями вращения. Стиральная машина дополнительно содержит крестовину основного барабана для соединения основного барабана с внешним валом для передачи движущей силы приводного средства к основному барабану и крестовину вспомогательного барабана для соединения вспомогательного барабана с внутренним валом для передачи движущей силы приводного средства к вспомогательному барабану, где две крестовины размещаются, чтобы быть способными вращаться независимо. 2 н. и 27 з.п. ф-лы, 80 ил.",
    "Изобретение относится к ударным инструментам, а именно к механизму гашения вибрации в механизированном ударном инструменте с линейным приводом наконечника. Инструмент содержит двигатель, вращающийся вал, качающийся элемент, механизм привода инструмента и противовес. Вращающийся вал располагается параллельно аксиальному направлению наконечника и приводится во вращение двигателем. Качающийся элемент поддерживается вращающимся валом для раскачивания в аксиальном направлении вращающегося вала за счет вращения вращающегося вала. Механизм привода инструмента соединяется с верхней частью качающегося элемента в вертикальном направлении. Механизм привода инструмента вызывает линейное перемещение в аксиальном направлении наконечника за счет качающего движения качающегося элемента и осуществляет линейный привод наконечника. Противовес расположен в области, находящейся выше нижнего концевого участка качающегося элемента в вертикальном направлении, которое пересекается с осью вращающегося вала. Нижний конец противовеса соединяется с нижним концевым участком качающегося элемента. Противовес вытянут вверх от соединения между противовесом и качающимся элементом и имеет точку поворота на вытянутом концевом участке. В результате уменьшается вибрация, возникающая в аксиальном направлении наконечника. 8 з.п. ф-лы, 17 ил.",
    "Изобретение относится к медицинской технике, а именно к хирургическим дрелям, более конкретно дрелям, имеющим системы управления. Хирургическая дрель содержит корпус, выдвигающуюся оболочку, втягивающий двигатель, двигатель дрели, пусковой переключатель, пусковой механизм дрели для вращения в прямом направлении и пусковой механизм дрели для вращения в обратном направлении. Выдвигающаяся оболочка расположена с возможностью скольжения в корпусе и перемещения между выдвинутым положением и втянутым положением. Втягивающий двигатель функционально соединен с выдвигающейся оболочкой для перемещения выдвигающейся оболочки между выдвинутым положением и втянутым положением. Двигатель дрели содержит вал двигателя, выполненный с возможностью соединения со сверлильным патроном и расположенный в выдвигающейся оболочке так, что вал двигателя может вращаться относительно выдвигающейся оболочки. Пусковой переключатель выполнен с возможностью обеспечения поступления электрической энергии селективно на втягивающий двигатель или двигатель дрели. Пусковой механизм дрели для вращения в прямом направлении функционально соединен с пусковым переключателем. Пусковой механизм дрели для вращения в обратном направлении функционально соединен с пусковым переключателем. Пусковой переключатель выполнен с возможностью приведения в действие двигателя дрели в первом направлении вращения, только когда задействован пусковой механизм дрели для вращения в прямом направлении. Пусковой переключатель выполнен с возможностью приведения в действие двигателя дрели в обратном направлении относительно первого направления вращения, только когда задействован пусковой механизм дрели для вращения в обратном направлении. Пусковой переключатель выполнен с возможностью приведения в действие втягивающего двигателя для перемещения втягивающейся оболочки из выдвинутого положения во втянутое положение, если одновременно задействованы пусковой механизм дрели для вращения в прямом направлении и пусковой механизм дрели для вращения в обратном направлении. Использование изобретения позволяет хирургу быстрее и с меньшим усилием втягивать сверло, тем самым избегая и/или снижая вероятность погружения дрели глубже, чем следует. 11 з.п. ф-лы, 2 ил.",
    "Изобретение относится к обрабатывающей установке и к способу обработки заготовок. Обрабатывающая установка и способ служат для сушки автомобильных кузовов с нанесенным покрытием. Обрабатывающая установка (100) для обработки заготовок (104), например для сушки автомобильных кузовов (106) с нанесенным покрытием, включает в себя технологическую камеру (110), транспортирующее устройство (114), обеспечивающее возможность подачи заготовок (104) к технологической камере (110), их отвода от технологической камеры (110) и/или перемещения через технологическую камеру (110) в направлении (112) перемещения, и тамбур (210), выполненный с возможностью протекания через него тамбурного газового потока, причем технологическая камера (110) включает в себя по меньшей мере один термический участок (204), обеспечивающий возможность нагрева заготовок (104) и/или поддержания их при температуре, повышенной по сравнению с температурой окружающей среды, и продувочное устройство для продувки продувочного участка (212) технологической камеры (11) потоком продувочного газа, причем тамбурный газовый поток после его протекания через тамбур (210) подается на заготовки (104) и направляется через заготовки (104) в качестве потока продувочного газа. Изобретения обеспечивают упрощение конструкции и возможность оптимизированной обработки заготовок. 2 н. и 28 з.п. ф-лы, 32 ил.",
    "Изобретение относится к медицине. Хирургический инструмент для резания и скрепления содержит рабочий орган, вал, который соединен с рабочим органом, и рукоятку. Рукоятка соединена с валом и содержит электрический двигатель постоянного тока и блок питания. Двигатель соединен с цепью привода на валу для приведения в действие цепи привода. Блок питания, по меньшей мере, одно зарядо-аккумулирующее устройство. Зарядно-аккумулирующее устройство соединено с двигателем постоянного тока для приведения в действие двигателя постоянного тока. В результате оператор имеет возможность лучше управлять как скоростью, так и мощностью двигателя. 2 н. и 16 з.п. ф-лы, 70 ил.",
    "Изобретение относится к технологическому картриджу. Технологический картридж содержит светочувствительный барабан; проявляющий валок; раму барабана; проявляющую раму, поддерживающую проявляющий валок, где проявляющая рама поддерживается рамой барабана с возможностью вращения вокруг ее оси вращения и выполнена с возможностью перемещения относительно рамы барабана между положением контакта, в котором проявляющий валок находится в контакте со светочувствительным барабаном, и разнесенным положением, в котором проявляющий валок расположен на расстоянии от светочувствительного барабана; подвижный элемент, имеющий первый принимающий усилие участок на одном конце подвижного элемента и второй принимающий усилие участок на другом конце подвижного элемента, причем второй принимающий усилие участок находится ближе к оси вращения, когда подвижный элемент находится в первом положении, чем, когда подвижный элемент находится во втором положении; и упругий элемент, выполненный с возможностью выталкивания подвижного элемента из второго положения в первое положение. Изобретение обеспечивает упрощение конструкции картриджа, облегчение обслуживания и повышение надежности конструкции. 3 н. и 24 з.п. ф-лы, 68 ил.",
    "Группа изобретений относится к картриджам для электрофотографического устройства формирования изображений. Сущность: основная сборка устройства включает в себя часть приложения усилия перемещения, часть зацепления основной сборки, приводящуюся в движение электродвигателем, имеющим приводной вал, часть приложения поворотного усилия, предусмотренную на приводном валу, и электрофотографический фоточувствительный барабан. При этом картридж является съемным с основной сборки без снятия электрофотографического фоточувствительного барабана с основной сборки. Картридж содержит: проявочный валик для проявления скрытого изображения, сформированного на электрофотографическом фоточувствительном барабане, вращаемый вокруг оси валика; вращающийся элемент для передачи поворотного усилия на проявочный валик; элемент муфты, вращаемый вокруг оси муфты посредством поворотного усилия, принимаемого от части зацепления основной сборки, при этом элемент муфты включает в себя часть приема поворотного усилия, зацепляемую с частью приложения поворотного усилия, для приема поворотного усилия от части зацепления основной сборки и часть передачи поворотного усилия для передачи поворотного усилия на вращающийся элемент; обойму, включающую в себя часть приема усилия перемещения, которая принимает усилие перемещения от части приложения усилия перемещения и поддерживает проявочный валик и вращающийся элемент. Технический результат: предотвращение неравномерности вращения проявочного валика, повышение скорости формирования изображения. 2 н. и 23 з.п. ф-лы, 78 ил."
]

ner_proccesed_texts = loaded_ner_model.pipe(text, disable=["tagger", "parser"])
threshold = 0.5

result = []
for doc in ner_proccesed_texts:
  cur_doc_result = {
      "document": doc.text,
      "tokens": [],
      "relations": []
  }
  for ent in doc.ents:
    cur_doc_result["tokens"].append({
        "text": ent.text,
        "start": ent.start_char,
        "end": ent.end_char,
        "token_start": ent.start,
        "token_end": ent.end,
        "entityLabel": ent.label_,
        "propertiesList": [],
        "commentsList": []
    })

  for name, proc in loaded_rel_model.pipeline:
    doc = proc(doc)

  for sent in doc.sents:
    for entity, entity_b in permutations(sent.ents, 2):
      rel_dict = doc._.rel.get((entity.start, entity_b.start))

      if rel_dict is not None:
        cur_max_relation = max(rel_dict.items(), key=lambda x: x[1])
        if (cur_max_relation[1] >= threshold):
          cur_doc_result["relations"].append({
              "child": entity_b.start,
              "head": entity.start,
              "relationLabel": cur_max_relation[0],
              "propertiesList": [],
              "commentsList": []
          })
  result.append(cur_doc_result)



In [6]:
import io
import json


with io.open("./model_results_10.json", mode="w", encoding="utf-8") as f:
    f.write(json.dumps(result, ensure_ascii=False))